# EDA of Garmin and supporting data

In [67]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # pandas dataframe
import os
from glob import glob
import json
import matplotlib.pyplot as plt
import seaborn as sns
garmin_base_dir = "../files/Garmin_20241403/DI_CONNECT"

pd.set_option('display.max_columns', 500)


# Building the health DataFrames

Combine json files into dataframes of healh data 
- Garmin daily health data files can be found in the Garmin-folder DI_CONNECT/DI-Connect-Aggregator, which contains json files starting with UDS.
- remove data from before 2020 (no watch), but there are some weird data rows?
- Also wellness data in DI_CONNECT/DI-Connect-Wellness

- Avtivities?

In [73]:
# Files
#  DI-Connect-Aggregator
#  DI-Connect-Welness
# Find activities?

# Function to load JSONs and combine them into a filtered DataFrame
def load_and_filter_json(base, file, substr, start_year=2020):
    all_dfs = []
    path = os.path.join(base, file)
    for root, _, files in os.walk(path):
        json_files = [f for f in sorted(files) if f.endswith('.json') and substr in f]
        for file in json_files:
            with open(os.path.join(root, file), 'r') as f:
                data = json.load(f)
#            df = pd.DataFrame(data) if isinstance(data, list) else pd.DataFrame([data])
            df = pd.json_normalize(data, sep='_')
            all_dfs.append(df)


    # Combining and filtering the data
    if all_dfs:
        full_df = pd.concat(all_dfs, ignore_index=True)

        # Check if 'metaData_calendarDate' column exists before trying to access it
        if 'metaData_calendarDate' in full_df.columns:
            full_df['calendarDate'] = pd.to_datetime(full_df['metaData_calendarDate'])
            return full_df
        elif 'calendarDate' in full_df.columns:
            full_df['calendarDate'] = pd.to_datetime(full_df['calendarDate'])
            return full_df
        else:
            print("No 'calendarDate' column found in data")
            return pd.DataFrame()  # Return empty DataFrame if column is missing
        
        full_df['calendarDate'] = pd.to_datetime(full_df['calendarDate'])
        full_df = full_df.loc[full_df['calendarDate'].dt.year >= start_year]#, columns_of_interest]
        return full_df
    return pd.DataFrame()  # Return empty DataFrame if no data was loaded
    

start_year = 2020

# DataFrames from json-files, combined
aggregator_df = load_and_filter_json(garmin_base_dir, "DI-Connect-Aggregator", 'UDS', start_year)
sleep_df = load_and_filter_json(garmin_base_dir, "DI-Connect-Wellness", 'sleepData', start_year)
bio_df = load_and_filter_json(garmin_base_dir, "DI-Connect-Wellness", 'userBioMetrics', start_year)

#df.loc[df['column_name'] == some_value]

### Aggregator_df containing most data

In [74]:
print(aggregator_df.columns)
aggregator_df

Index(['userProfilePK', 'calendarDate', 'uuid', 'durationInMilliseconds',
       'consumedKilocalories', 'remainingKilocalories',
       'includesCalorieConsumedData', 'version', 'totalKilocalories',
       'activeKilocalories', 'burnedKilocalories', 'totalDistanceMeters',
       'includesWellnessData', 'includesActivityData', 'bmrKilocalories',
       'wellnessKilocalories', 'wellnessTotalKilocalories',
       'wellnessActiveKilocalories', 'totalSteps', 'dailyStepGoal',
       'wellnessDistanceMeters', 'wellnessStartTimeGmt', 'wellnessEndTimeGmt',
       'wellnessStartTimeLocal', 'wellnessEndTimeLocal', 'highlyActiveSeconds',
       'activeSeconds', 'moderateIntensityMinutes', 'vigorousIntensityMinutes',
       'floorsAscendedInMeters', 'floorsDescendedInMeters',
       'userIntensityMinutesGoal', 'userFloorsAscendedGoal', 'minHeartRate',
       'maxHeartRate', 'restingHeartRate', 'currentDayRestingHeartRate',
       'restingHeartRateTimestamp', 'includesSingleMeasurement',
       'in

,userProfilePK,calendarDate,uuid,durationInMilliseconds,consumedKilocalories,remainingKilocalories,includesCalorieConsumedData,version,totalKilocalories,activeKilocalories,burnedKilocalories,totalDistanceMeters,includesWellnessData,includesActivityData,bmrKilocalories,wellnessKilocalories,wellnessTotalKilocalories,wellnessActiveKilocalories,totalSteps,dailyStepGoal,wellnessDistanceMeters,wellnessStartTimeGmt,wellnessEndTimeGmt,wellnessStartTimeLocal,wellnessEndTimeLocal,highlyActiveSeconds,activeSeconds,moderateIntensityMinutes,vigorousIntensityMinutes,floorsAscendedInMeters,floorsDescendedInMeters,userIntensityMinutesGoal,userFloorsAscendedGoal,minHeartRate,maxHeartRate,restingHeartRate,currentDayRestingHeartRate,restingHeartRateTimestamp,includesSingleMeasurement,includesContinuousMeasurement,includesAllDayPulseOx,includesSleepPulseOx,source,minAvgHeartRate,maxAvgHeartRate,lowestSpo2Value,latestSpo2Value,latestSpo2ValueReadingTimeGmt,latestSpo2ValueReadingTimeLocal,allDayStress_userProfilePK,allDayStress_calendarDate,allDayStress_aggregatorList,bodyBattery_userProfilePK,bodyBattery_calendarDate,bodyBattery_chargedValue,bodyBattery_drainedValue,bodyBattery_bodyBatteryVersion,bodyBattery_bodyBatteryStatList,hydration_userProfilePK,hydration_calendarDate,hydration_valueInML,hydration_activityIntakeInML,hydration_goalInML,hydration_capped,hydration_sweatLossInML,hydration_adjustedGoalInML,hydration_lastEntryTimestampLocal,respiration_userProfilePK,respiration_calendarDate,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,respiration_latestRespirationValue,respiration_latestRespirationTimeGMT,averageSpo2Value,netCalorieGoal,dailyTotalFromEpochData,restingCaloriesFromActivity,isVigorousDay
0,86607424,2006-01-01,0a391461192c46aca6edc90565a1b893,0,0.0,0.0,True,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,86607424,2011-05-28,e4087f3e7785436198820bb49e4ba85a,0,NaN,1254.0,False,0,1254.0,1254.0,1254.0,21200.0,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,86607424,2011-08-21,a225b5de26d74a9c801eaa679893dbd5,0,NaN,1269.0,False,0,1269.0,1269.0,1269.0,21100.0,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,86607424,2011-09-25,b611fee6d6f94c82bb7b17c4ebfb2666,0,NaN,1291.0,False,0,1291.0,1291.0,1291.0,21100.0,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,86607424,2012-10-07,88ef2e8dabf2426a981c252fd6358288,0,NaN,1438.0,False,0,1438.0,1438.0,1438.0,21211.0,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,86607424,2024-03-10,3ac1beeb93b143648883

In [75]:
# chose only intersting columns

focus_columns_agg = ['calendarDate', 'totalKilocalories', 'activeKilocalories', 'allDayStress_aggregatorList', 
                       'includesWellnessData', 'includesActivityData',
                       'totalSteps', 'highlyActiveSeconds', 'moderateIntensityMinutes', 'vigorousIntensityMinutes', 
                       'highlyActiveSeconds', 'userIntensityMinutesGoal', 
                       'minHeartRate', 'maxHeartRate', 'restingHeartRate', 'minAvgHeartRate', 'maxAvgHeartRate',
                       'bodyBattery_chargedValue', 'bodyBattery_drainedValue', 'bodyBattery_bodyBatteryStatList',
                         'restingCaloriesFromActivity', 'isVigorousDay']

focus_agg_df = aggregator_df[focus_columns_agg]

#focus_agg_df =  aggregator_df.loc[aggregator_df['calendarDate'].dt.year<=2020, focus_columns_agg]
focus_agg_df

,calendarDate,totalKilocalories,activeKilocalories,allDayStress_aggregatorList,includesWellnessData,includesActivityData,totalSteps,highlyActiveSeconds,moderateIntensityMinutes,vigorousIntensityMinutes,highlyActiveSeconds,userIntensityMinutesGoal,minHeartRate,maxHeartRate,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,bodyBattery_chargedValue,bodyBattery_drainedValue,bodyBattery_bodyBatteryStatList,restingCaloriesFromActivity,isVigorousDay
0,2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-05-28,1254.0,1254.0,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-08-21,1269.0,1269.0,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-09-25,1291.0,1291.0,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-10-07,1438.0,1438.0,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,2024-03-10,2038.0,581.0,"[{'type': 'TOTAL', 'averageStressLevel': 29, '...",True,True,13169.0,3086.0,3.0,25.0,3086.0,500.0,50.0,148.0,60.0,51.0,146.0,52.0,11.0,"[{'bodyBatteryStatType': 'HIGHEST', 'statsValu...",24.0,True
1375,2024-03-11,2231.0,774.0,"[{'type': 'TOTAL', 'averageStressLevel': 20, '...",True,True,12551.0,2202.0,5.0,38.0,2202.0,500.0,54.0,161.0,60.0,55.0,160.0,79.0,68.0,"[{'bodyBatteryStatType': 'HIGHEST', 'statsValu...",38.0,True
1376,2024-03-12,2762.0,1305.0,"[{'type': 'TOTAL', 'averageStressLevel': 20, '...",True,True,11733.0,3126.0,44.0,98.0,3126.0,500.0,55.0,147.0,60.0,56.0,146.0,42.0,68.0,"[{'bodyBatteryStatType': 'HIGHEST', 'statsValu...",170.0,True
1377,2024-03-13,2372.0,915.0,"[{'type': 'TOTAL', 'averageStressLevel': 29, '...",True,True,16581.0,4380.0,19.0,49.0,4380.0,500.0,55.0,148.0,60.0,55.0,146.0,65.0,53.0,"[{'bodyBatteryStatType': 'HIGHEST', 'statsValu...",65.0,True


### Sleep_df, hours of sleep I guess

In [76]:
sleep_df.head()

,sleepStartTimestampGMT,sleepEndTimestampGMT,calendarDate,sleepWindowConfirmationType,retro,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,unmeasurableSeconds,averageRespiration,lowestRespiration,highestRespiration,spo2SleepSummary_userProfilePk,spo2SleepSummary_deviceId,spo2SleepSummary_sleepMeasurementStartGMT,spo2SleepSummary_sleepMeasurementEndGMT,spo2SleepSummary_alertThresholdValue,spo2SleepSummary_averageSPO2,spo2SleepSummary_averageHR,spo2SleepSummary_lowestSPO2
0,2020-06-17T21:00:00.0,2020-06-18T04:00:00.0,2020-06-18,UNCONFIRMED,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-06-18T21:25:00.0,2020-06-19T03:46:00.0,2020-06-19,ENHANCED_CONFIRMED_FINAL,False,2880.0,15600.0,3720.0,660.0,0.0,16.0,6.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-06-19T19:58:00.0,2020-06-20T04:14:00.0,2020-06-20,ENHANCED_CONFIRMED_FINAL,False,3660.0,16020.0,8100.0,840.0,1140.0,16.0,7.0,21.0,86607424.0,3.334930e+09,2020-06-19T20:52:00.0,2020-06-20T04:00:00.0,0.0,93.0,60.0,84.0
3,2020-06-20T21:56:00.0,2020-06-21T04:52:00.0,2020-06-21,ENHANCED_CONFIRMED_FINAL,False,3120.0,15180.0,5640.0,1020.0,0.0,15.0,7.0,18.0,86607424.0,3.334930e+09,2020-06-20T21:57:00.0,2020-06-21T04:00:00.0,NaN,95.0,58.0,93.0
4,2020-06-21T21:09:00.0,2020-06-22T04:55:00.0,2020-06-22,ENHANCED_CONFIRMED_FINAL,False,3120.0,17400.0,7320.0,120.0,0.0,16.0,6.0,21.0,86607424.0,3.334930e+09,2020-06-21T21:10:00.0,2020-06-22T04:00:00.0,0.0,96.0,60.0,87.0


In [78]:
print(sleep_df.columns)
focus_columns_well = ['sleepStartTimestampGMT', 'sleepEndTimestampGMT', 'calendarDate',
                     'deepSleepSeconds', 'lightSleepSeconds', 'remSleepSeconds', 'awakeSleepSeconds','unmeasurableSeconds',
                     ]

Index(['sleepStartTimestampGMT', 'sleepEndTimestampGMT', 'calendarDate',
       'sleepWindowConfirmationType', 'retro', 'deepSleepSeconds',
       'lightSleepSeconds', 'remSleepSeconds', 'awakeSleepSeconds',
       'unmeasurableSeconds', 'averageRespiration', 'lowestRespiration',
       'highestRespiration', 'spo2SleepSummary_userProfilePk',
       'spo2SleepSummary_deviceId',
       'spo2SleepSummary_sleepMeasurementStartGMT',
       'spo2SleepSummary_sleepMeasurementEndGMT',
       'spo2SleepSummary_alertThresholdValue', 'spo2SleepSummary_averageSPO2',
       'spo2SleepSummary_averageHR', 'spo2SleepSummary_lowestSPO2'],
      dtype='object')


In [80]:
focus_sleep_df = sleep_df[focus_columns_well]
focus_sleep_df

,sleepStartTimestampGMT,sleepEndTimestampGMT,calendarDate,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,unmeasurableSeconds
0,2020-06-17T21:00:00.0,2020-06-18T04:00:00.0,2020-06-18,NaN,NaN,NaN,NaN,NaN
1,2020-06-18T21:25:00.0,2020-06-19T03:46:00.0,2020-06-19,2880.0,15600.0,3720.0,660.0,0.0
2,2020-06-19T19:58:00.0,2020-06-20T04:14:00.0,2020-06-20,3660.0,16020.0,8100.0,840.0,1140.0
3,2020-06-20T21:56:00.0,2020-06-21T04:52:00.0,2020-06-21,3120.0,15180.0,5640.0,1020.0,0.0
4,2020-06-21T21:09:00.0,2020-06-22T04:55:00.0,2020-06-22,3120.0,17400.0,7320.0,120.0,0.0
...,...,...,...,...,...,...,...,...
1359,2024-03-09T22:12:00.0,2024-03-10T07:39:00.0,2024-03-10,9780.0,18060.0,5880.0,300.0,0.0
1360,2024-03-10T18:04:00.0,2024-03-11T04:45:00.0,2024-03-11,4560.0,22500.0,9420.0,1980.0,0.0
1361,2024-03-11T20:14:00.0,2024-03-12T04:25:00.0,2024-03-12,2580.0,17880.0,8160.0,840.0,0.0
1362,2024-03-12T22:03:00.0,2024-03-13T02:10:00.0,2024-03-13,5220.0,4680.0,2760.0,240.0,1920.0


### bio_df containing vo2max

In [81]:
vo2max_df = bio_df[['calendarDate', 'vo2MaxRunning']]
vo2max_df

,calendarDate,vo2MaxRunning
0,2020-06-18 00:00:00.000,42.0
1,2020-06-18 00:00:00.000,42.0
2,2020-06-18 05:30:15.000,NaN
3,2020-06-18 10:33:40.140,NaN
4,2020-06-18 10:36:33.958,NaN
...,...,...
1362,2024-03-07 00:00:00.000,51.0
1363,2024-03-09 00:00:00.000,51.0
1364,2024-03-10 00:00:00.000,51.0
1365,2024-03-12 00:00:00.000,50.0


In [86]:
focus_agg_df[['allDayStress_aggregatorList']]

,allDayStress_aggregatorList
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1374,"[{'type': 'TOTAL', 'averageStressLevel': 29, '..."
1375,"[{'type': 'TOTAL', 'averageStressLevel': 20, '..."
1376,"[{'type': 'TOTAL', 'averageStressLevel': 20, '..."
1377,"[{'type': 'TOTAL', 'averageStressLevel': 29, '..."


## Load activities as well